# Query IP reputation

## Introduction

- UUID: **39a029be-e4d7-40da-88b2-1d0bf9417ad0**
- Started from [issue 12](https://github.com/MISP/misp-playbooks/issues/12)
- State: **Published** : demo version with **output**
- Purpose: A playbook to query for the reputation of one or more IPs. 
    - It combines the reputation scores from **VirusTotal**, **Shodan**, **Greynoise** and **AbuseIPDB** into one MISP report. 
    - The playbook adds the known associated **domains**, the **abuse contacts** and the geo information from **MMDB**.
    - All information is added to a MISP event, summarised and send to **Mattermost** and **TheHive**.
- Tags: [ "ip", "reputation", "abuse", "whois" ]
- External resources: **abuse_finder**, **DNS**, **MMDB**, **Shodan**, **Greynoise**, **VirusTotal**, **AbuseIPDB**, **Mattermost, TheHive**
- Target audience: **SOC**, **CSIRT**, **CTI**

# Playbook

- **Query IP reputation**
    - Introduction
- **Preparation**
    - PR:1 Initialise environment
    - PR:2 Verify MISP modules
    - PR:3 Load helper functions
    - PR:4 Set helper variables
    - PR:5 Provide the list of IPs
    - PR:6 MISP event details
    - PR:7 Setup MISP event link
- **Investigate**
    - IN:1 Context details for the IPs
    - IN:2 Add the IPs to the event
- **Correlation**
    - CR:1 Correlation with MISP events
    - CR:2 Correlation with MISP feeds
- **Enrichment**
    - ER:1 Enrich with DNS information
    - ER:2 Enrich with abuse information
    - ER:3 Enrich with information from VirusTotal
    - ER:4 Enrich with information from Shodan
    - ER:5 Get reputation from Greynoise
    - ER:6 Get geo information from MMDB
    - ER:7 Enrich with information from AbuseIPDB
    - ER:8 Investigation and enrichment report
- **Summary**
    - EN:1 MISP indicators
    - EN:2 Create the summary of the playbook
    - EN:3 Print the summary
    - EN:4 Display a map of the IPs
    - EN:5 Send a summary to Mattermost    
    - EN:6 Send an alert to TheHive
    - EN:7 End of the playbook
- External references
- Technical details

# Preparation

## PR:1 Initialise environment

This section **initialises the playbook environment** and loads the required Python libraries. 

The credentials for MISP (**API key**) and other services are loaded from the file `keys.py` in the directory **vault**. A [PyMISP](https://github.com/MISP/PyMISP) object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least :

```
misp_url="<MISP URL>"                  # The URL to our MISP server
misp_key="<MISP API KEY>"              # The MISP API key
misp_verifycert=<True or False>        # Indicate if PyMISP should attempt to verify the certificate or ignore errors
urlscan_url="https://urlscan.io/api/v1/search"
urlscan_apikey="<URLSCAN API KEY>"
mattermost_playbook_user="<MATTERMOST USER>"
mattermost_hook="<MATTERMOST WEBHOOK>"
thehive_url="<THEHIVE URL>"
thehive_key="<THEHIVE API KEY>"
virustotal_apikey="<VIRUSTOTAL_APIKEY>"
shodan_apikey="<SHODAN_APIKEY>"
greynoise_apikey="<GREYNOISE_APIKEY>"
abuseipdb_apikey=<ABUSEIPDB_APIKEY>"
```

In [ ]:
# Initialise Python environment
import urllib3
import sys
import json
import base64
import uuid
import time
import re
from prettytable import PrettyTable, MARKDOWN
from IPython.display import Image, display, display_markdown, HTML
from datetime import date
import requests
from pymisp import *
from pymisp.tools import GenericObjectGenerator
from abuse_finder import domain_abuse, ip_abuse, email_abuse, url_abuse
import textwrap
from datetime import datetime

# Used for -optionally- drawing the maps.
import folium

# Load the credentials
sys.path.insert(0, "../vault/")
from keys import *
if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
misp_headers = {"Authorization": misp_key,  "Content-Type": "application/json", "Accept": "application/json"}
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n".format(misp_url))

## PR:2 Verify MISP modules

This playbook uses the MISP modules to obtain additional correlation or enrichment information. [MISP modules](https://github.com/MISP/misp-modules) are autonomous modules that can be used to extend MISP for new services such as expansion, import and export. The modules are written in Python 3 following a simple API interface. The objective is to ease the extensions of MISP functionalities without modifying core components. The API is available via a simple REST API which is independent from MISP installation or configuration.

In [ ]:
# Where can we find the local MISP Module server? You can leave this to the default setting in most cases.
misp_modules_url = "http://127.0.0.1:6666"

# How long do we wait between queries when using the MISP modules (API rate limiting of external service such as VirusTotal)
misp_modules_wait = 3

# Initiliasation
misp_modules = {}
misp_modules_headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}
misp_modules_in_use = ["reversedns", "virustotal_public", "shodan"]
# Code block to query the MISP module server and check if our modules are enabled
res = requests.get("{}/modules".format(misp_modules_url), headers=misp_modules_headers)
for module in res.json():
    for module_requested in misp_modules_in_use:
        if module.get("name", False) == module_requested:
            misp_modules[module_requested] = {"enabled": True, "input": module.get("mispattributes").get("input")}
            print("Found the \033[92m{}\033[90m MISP module (Accepted input: {}).".format(module_requested, misp_modules[module_requested]["input"]))

## PR:3 Load helper functions

The next cell contains **helper functions** that are used in this playbook. 

Instead of distributing helper functions as separate Python files this playbook includes all the required code as one code cell. This makes portability of playbooks between instances easier. The downside is that functions defined in this playbook need to be defined again in other playbooks, which is not optimal for code re-use. For this iteration of playbooks it is chosen to include the code in the playbook (more portability), but you can easily create one "helper" file that contains all the helper code and then import that file in each playbook (for example by adding to the previous cell `from helpers import *`). Note that the graphical workflow image is included as an external image. A missing image would not influence the further progress of the playbook.

In [ ]:
def pb_get_misp_tags(tags=[], local_tags=[]):
    '''
    Get a list of MISP tags based on a Python list

    :param misp: MISP object
    :param object_template: which object template to return
    '''
    misp_tags = []
    for el in tags:
        t = MISPTag()
        t.name = el
        t.local = False
        misp_tags.append(t)

    for el in local_tags:
        t = MISPTag()
        t.name = el
        t.local = True
        misp_tags.append(t)
    return misp_tags



def get_playbook_result(value, source, category, res_type):
    '''
    Lookup the playbook results for a list of entries
    
    : param value: The IP address under investigation
    : param source: Which source provided the enrichment?
    : param category: Enrichment category
    : param res_type: Enrichment type
    '''
    return_value = []
    for el in playbook_results[value]:
        if el["source"] == source and el["category"] == category and el["type"] == res_type:
            if el["enriched"] not in return_value:
                if el["enriched"] is list:
                    for list_item in el["enriched"]:
                        return_value.append(str(list_item))
                else:
                    return_value.append(str(el["enriched"]))
    return_value = ' '.join(return_value).strip()
    if len(return_value) < 1:
        return_value = "-"
    return return_value



def pb_add_enrichment(playbook_results, field, entry, key, value):
    '''
    Add an enrichment (or correlation) entry but first check that the value is not already there
    
    : param playbook_results: all the enrichment results
    : param field
    : param entry
    : param key
    : param value
    '''
    skip_field = False
    for existing_entry in playbook_results.get(field, []):
        if existing_entry.get(key, False) == value:
            skip_field = True
            print(" Not adding to playbook results because of duplicate. Already added via {}".format(existing_entry.get("source", False)))
    if not skip_field:
        if field in playbook_results:
            playbook_results[field].append(entry)
        else:
            playbook_results[field] = [entry]
    return playbook_results


print("\033[92mHelper functions loaded\033[90m.\n".format(misp_url))

## PR:4 Set helper variables

This cell contains **helper variables** for this playbook.

In [ ]:
# Dictionary to playbook results and some of the core objects that are created
playbook_results = {}
case_objects = {}

# A set of regular expressions that we use to determine the attribute type in a warninglist
regular_expressions = {"sha256": "^[a-fA-F0-9]{64}$",
                       "md5": "^[a-fA-F0-9]{32}$",
                       "hostname": "^[a-zA-Z0-9.\-_]+\.[a-zA-Z]{2,}$",
                       "sha1": "^[a-fA-F0-9]{40}$",
                       "url": "^(http|https):\/\/[-a-zA-Z0-9-]{2,256}\.[-a-zA-Z0-9-]{2,256}",
                       "ip-src": "[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}",
                       "ip-dst": "[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}"
                      }

## PR:5 Provide the list of IPs

Add the **IPs** that you want to investigate with this playbook to the variable `query`. You can provide one IP or a list of IPs. You can supply an IPv4 or IPv6 address but remember that not all enrichment services provide sufficient results for IPv6.

In [ ]:
# Provide one or more IPs
query = ["91.219.237.56", "185.194.93.128"]

In [ ]:
# Simple function to make a list. Makes it more consistent to work with the user input in the next cells.
if type(query) == str:
    query =[query]
print("The playbook will do the IP reputation query for \033[92m{}\033[90m\n".format(query))

## PR:6 MISP event details

### Event title

In this playbook we create a **new** MISP event with title **IP reputation _IPlist_**. You get the chance to override this default title but remember that it is good practice to choose a self-explanatory **event title**. This event title is shown in the MISP event index and should provide you the necessary information what the event is about. You should avoid using generic event titles. Read the [Best Practices in Threat Intelligence](https://www.misp-project.org/best-practices-in-threat-intelligence.html) for further guidance.

### Contexualisation

This playbook adds event contexualisation via the **tags** that are defined in `event_additional_global_tags` (for *global* tags) and `event_additional_local_tags` (for *local* tags). As a reminder, whereas *global* tags remain attached to the events that you share with your community, the *local* tags are not shared outside your organisation. It's also a good idea to primarily use tags that are part of a [taxonomy](https://github.com/MISP/misp-taxonomies), this allows you to make the contexualisation more portable accross multiple MISP instances.

In this playbook the list of tags is build via one of the helper functions `pb_get_misp_tags`. This function takes two arguments, first a list of tags to convert as *global* tags, and secondly a list of tags to convert as *local* tags. It then returns a Python list of MISPTag objects.

### Traffic Light Protocol

The default **TLP** for this event is **<span style='color:#FFBF00'>tlp:amber</span>**. The Traffic Light Protocol (TLP) facilitates sharing of potentially sensitive information and allows for more effective collaboration. TLP is a set of four standard labels to indicate the sharing boundaries to be applied by the recipients. TLP is always set by the creator of information. You can find more information at [FIRST](https://www.first.org/tlp/). You can specify the TLP via `event_tlp`.

### MISP Galaxies

This playbook can also add MISP galaxies to the event with the variable `event_galaxies`. You can also leave the list empty if you do not want to add galaxies in this stage of the investigation.

### MISP distribution, threat level and analysis level

Optionally you can specifiy a MISP **distribution** (with `event_distribution`), **threat level** (with `event_threat_level_id`) or **analysis state** (with `event_analysis`). The event **date** is set to today via `event_date`.

If you cannot remember the options for distribution, threat level or the analysis state then use the next cell to guide you. This cell is set as **raw**. If you **change its type to code** and execute the cell you get an overview of the options available for creating a MISP event.

## PR:7 Setup MISP event link

By default the playbook will generate a **title** with a prefix and the IPs you want to investigate. You can override this event title with the variable `event_title`. If you leave `event_title` empty the playbook will generate the MISP event title for you.

In [ ]:
# Provide the event title for a new event. Leave blank for the playbook to auto generate one
event_title = ""

# Prefix for auto generate event title
event_title_default_prefix = "IP reputation"

# Optionally, you can change TLP, add additional event (local and global) tags, threatlevel, analysis state or distribution level
event_tlp = "tlp:amber"

# Event context
event_additional_global_tags = []                                 # This needs to be a Python list
event_additional_local_tags = ["workflow:state=\"incomplete\""]   # This needs to be a Python list

# Event galaxies
event_galaxies = ["misp-galaxy:mitre-attack-pattern=\"IP Addresses - T1590.005\""]

# Additional MISP event settings
event_threat_level_id = ThreatLevel.low
event_analysis = Analysis.ongoing
event_distribution = Distribution.your_organisation_only
event_date = date.today()

### Create MISP event

The next cell **creates the MISP event** and stores the event object in `misp_event`.

In [ ]:
# Code block to create the event or add data to an existing event
event_title = event_title.strip()

if not(len(event_title) > 0):
    query_sample = ""
    for key in query:
        query_sample = "{} {}".format(key, query_sample)
    event_title = "{} for {}".format(event_title_default_prefix, query_sample)

# Construct the event tags
event_additional_global_tags.append(event_tlp)
if len(event_galaxies) > 0:
    event_additional_global_tags.append(event_galaxies)
event_tags = pb_get_misp_tags(event_additional_global_tags, event_additional_local_tags)

# Create the PyMISP object for an event
event = MISPEvent()
event.info = event_title
event.distribution = event_distribution
event.threat_level_id = event_threat_level_id
event.analysis = event_analysis
event.set_date(event_date)

# Create the MISP event on the server side
misp_event = misp.add_event(event, pythonify=True)
print("Continue the playbook with the new \033[92mcreated\033[90m MISP event ID {} with title \033[92m{}\033[90m and UUID {}".format(misp_event.id, misp_event.info, misp_event.uuid))
for tag in event_tags:
    if len(tag) > 0:
        misp.tag(misp_event.uuid, tag, local=tag.local)
        print(" \033[92mAdded\033[90m event tag {}".format(tag))

# Investigate

## IN:1 Context details for the IPs

Set the contextualisation (tags) for the IPs with `attribute_tags`. These tags are added to the IPs, and their corresponding reverse DNS entries.

In [ ]:
attribute_tags = pb_get_misp_tags(["PAP:GREEN", "course-of-action:passive=\"discover\""])

## IN:2 Add the IPs to the event

Add the IPs to the MISP event.

In [ ]:
print("Add IPs to MISP event.")
for ip in query:
    domain_ip_object = MISPObject("domain-ip")
    domain_ip_object.add_attribute("ip", ip, tags=attribute_tags, comment="Added by playbook")
    result = misp.add_object(misp_event.uuid, domain_ip_object, pythonify=True)
    if not "errors" in result:
        case_objects[ip] = result
        playbook_results[ip] = []
        print(" Added domain-ip object for \033[92m{}\033[90m with UUID {}".format(ip, result.uuid))
    else:
        print(result)

# Correlation

## CR:1 Correlation with MISP events

Search the **MISP server** for events that match with one of the IPs you specified in `query`. A summary is also shown at the end of the playbook.

Only **published** events (`correlation_published`) and attributes that have the **to_ids** flag (`correlation_to_ids`) are considered. You can limit the number of results (`correlation_limit`) and limit the search with tags (`correlation_match_tags`).

In [ ]:
# Only query for published MISP events
correlation_published = None #True

# Only consider those values that have the to_ids field set to True
correlation_to_ids = True

# Limit the returned results. Use "None" to include all results
correlation_limit = None

# Only return results corresponding with these tags
correlation_match_tags = ["tlp:amber", "tlp:white"]

In [ ]:
print("Search for correlating MISP events")
# Code block to query MISP and find the correlations
if len(query) > 0:
    search_match = misp.search("attributes", to_ids=correlation_to_ids, value=query, tags=correlation_match_tags,
                                   published=correlation_published, limit=correlation_limit, pythonify=True)
    if len(search_match) > 0:
        for attribute in search_match:
            if attribute.Event.id != misp_event.id:   # Skip the event we just created for this playbook
                print(" Found \033[92m{}\033[90m in event \033[92m{}\033[90m - \033[92m{}\033[90m".format(attribute.value, attribute.event_id, attribute.Event.info))
                entry = {"source": "MISP", "category": attribute.category, "type": attribute.type, "event_id": attribute.Event.id, "event_info": attribute.Event.info, "enriched": attribute.value}
                playbook_results = pb_add_enrichment(playbook_results, attribute.value, entry, "event_id", attribute.Event.id)
print("Finished searching for correlations")

### MISP events correlation table

The correlation results are now stored in `playbook_results`. Execute the next cell to display them in a table format. The table is also included in the summary for Mattermost and TheHive.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Category", "Type", "Event", "Event ID"]
table.align["Value"] = "l"
table.align["Category"] = "l"
table.align["Type"] = "l"
table.align["Event"] = "l"
table.align["Event ID"] = "l"
table._max_width = {"Event": 50}
for ip in playbook_results:
    for match in playbook_results[ip]:
        if match["source"] == "MISP":
            table.add_row([match["source"], ip, match["category"], match["type"], match["event_info"], match["event_id"]])
print(table.get_string(sortby="Value"))
table_mispevents = table

## CR:2 Correlation with MISP feeds

Search the **MISP feeds** for events that match with one of the IPs you specified in `query`. The results highly depend on the feeds you have enabled. For good results, it's advised you enable at least
- **Tor exit nodes**
- **Feodo IP Blocklist**

Be sure to check if other [MISP feeds](https://www.misp-project.org/feeds/) provide valuable results for your environment as well. Note that the correlation lookup in the MISP feeds does not return the name of the MISP event, it returns the UUID of the event as title.

In [ ]:
print("Search in MISP feeds")
if len(query) > 0:
    misp_cache_url = "{}/feeds/searchCaches/".format(misp_url)
    match = False
    for ip in query:
        # Instead of GET, use POST (https://github.com/MISP/MISP/issues/7478)
        cache_results = requests.post(misp_cache_url, headers=misp_headers, verify=misp_verifycert, json={"value": ip})
        for result in cache_results.json():
            if "Feed" in result:
                match = True
                print("Found \033[92m{}\033[90m in \033[92m{}\033[90m".format(ip, result["Feed"]["name"]))
                for match in result["Feed"]["direct_urls"]:
                    entry = {"source": "Feeds", "feed_name": result["Feed"]["name"], "match_url": match["url"]}
                    playbook_results = pb_add_enrichment(playbook_results, ip, entry, "match_url", match["url"])
print("Finished searching in MISP feeds")

### MISP feed correlations table

The correlation results are now stored in `playbook_results`. Execute the next cell to display them in a table format. The table is also included in the summary for Mattermost and TheHive.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Feed", "URL"]
table.align["Value"] = "l"
table.align["Feed"] = "l"
table.align["Feed URL"] = "l"
table._max_width = {"Feed": 50}
for ip in playbook_results:
    for match in playbook_results[ip]:
        if match["source"] == "Feeds":
            table.add_row([match["source"], ip, match["feed_name"], match["match_url"]])
print(table.get_string(sortby="Value"))
table_mispfeeds = table

# Enrichment

## ER:1 Enrich with DNS information

Query the ReverseDNS module and add the **reverse DNS** results to the previously created `domain-ip` object(s). This module uses the default DNS resolver defined in the MISP module (8.8.8.8) but you can specify your own DNS server with `module_dnsserver`.

In [ ]:
# DNS server to use. Leave blank to use 8.8.8.8 (default MISP module)
module_dnsserver = ""

module_name = "reversedns"
attribute_type = "ip-dst"

print("Start {}.".format(module_name))
for value in query:
    module_comment = "From {} for {}".format(module_name, value)    
    data = {
        attribute_type: f"{value}",
        "module": module_name
    }
    if len(module_dnsserver) > 0:
        data["config"] = { "nameserver": module_dnsserver }
    print("Query \033[92m{}\033[90m".format(value))    
    result = requests.post("{}/query".format(misp_modules_url), headers=misp_modules_headers, json=data)
    if "results" in result.json() and len(result.json()["results"]) > 0:
        result_json = result.json()["results"]
        for entry in result_json:
            if "values" in entry:
                for module_value in entry["values"]:
                    # Avoid hostname validation errors when ending in a "."
                    if module_value.endswith('.'):
                        module_value = module_value[:-1]
                    ip_object = misp.get_object(case_objects[value].uuid, pythonify=True)
                    ip_object.add_attribute("hostname", module_value, comment=module_comment, tags=attribute_tags)
                    ip_object_misp = misp.update_object(ip_object, pythonify=True)
                    if not "errors" in ip_object_misp:
                        print(" Got \033[92m{}\033[90m".format(module_value))
                        entry = {"source": module_name, "category": "Network activity", "type": "hostname", "enriched": module_value}
                        playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", module_value)

print("Finished {}.".format(module_name))

## ER:2 Enrich with abuse information

Use [abuse_finder](https://github.com/certsocietegenerale/abuse_finder) information to get the most appropriate contact details for **abuse reports** and add them as MISP attributes (either `whois-registrant-name` or `whois-registrant-email`).

Note that the current Python module for abuse_finder does not yet [support IPv6](https://github.com/certsocietegenerale/abuse_finder/pull/5).

In [ ]:
# Relationship type for abuse records
relationtype = "associated-with"

module_name = "abuse_finder"

# Lookup abuse details for the IPs
print("Start {}.".format(module_name))
for value in query:
    ipv6_pattern = r'^(([0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,7}:|([0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,5}(:[0-9a-fA-F]{1,4}){1,2}|([0-9a-fA-F]{1,4}:){1,4}(:[0-9a-fA-F]{1,4}){1,3}|([0-9a-fA-F]{1,4}:){1,3}(:[0-9a-fA-F]{1,4}){1,4}|([0-9a-fA-F]{1,4}:){1,2}(:[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:((:[0-9a-fA-F]{1,4}){1,6})|:((:[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(:[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(ffff(:0{1,4}){0,1}:){0,1}((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])|([0-9a-fA-F]{1,4}:){1,4}:((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9]))$'
    if re.match(ipv6_pattern, value) is not None:
        print("Current abuse_finder module does not support IPv6.")
        continue

    module_comment = "From {} for {}".format(module_name, value)
    print("Trying \033[92m{}\033[90m".format(value))
    details = ip_abuse(value)
    if "names" in details:
        attribute_category = "Attribution"
        attribute_type = "whois-registrant-name"
        for name in details["names"]:
            print(" Got \033[92m{}\033[90m".format(name))
            entry = {"source": module_name, "category": attribute_category, "type": attribute_type, "enriched": name}
            playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", name)
            attribute = MISPAttribute()
            attribute.value = name
            attribute.to_ids = False
            attribute.category = attribute_category
            attribute.type = attribute_type
            attribute.disable_correlation = True
            attribute.comment = "{}. Information for {}".format(module_comment, value)
            attribute_misp = misp.add_attribute(misp_event.uuid, attribute, pythonify=True)
            if not "errors" in attribute_misp:
                misp.add_object_reference(case_objects[value].add_reference(attribute_misp.uuid, relationtype))
            else:
                print(attribute_misp)
    if "abuse" in details:
        attribute_category = "Attribution"
        attribute_type = "whois-registrant-email"
        for name in details["abuse"]:
            print(" Got \033[92m{}\033[90m".format(name))
            entry = {"source": module_name, "category": "Attribution", "type": "whois-registrant-email", "enriched": name}
            playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", name)
            attribute = MISPAttribute()
            attribute.value = name
            attribute.to_ids = False
            attribute.category = attribute_category
            attribute.type = attribute_type
            attribute.disable_correlation = True
            attribute.comment = "{}. Information for {}".format(module_comment, value)
            attribute_misp = misp.add_attribute(misp_event.uuid, attribute, pythonify=True)
            if not "errors" in attribute_misp:
                misp.add_object_reference(case_objects[value].add_reference(attribute_misp.uuid, relationtype))
            else:
                print(attribute_misp)

print("Finished {}.".format(module_name))

## ER:3 Enrich with information from VirusTotal

Next we query [VirusTotal](https://www.virustotal.com/gui/). First we use the MISP module because it returns MISP attributes and objects that can then be easily added to the event.

Then we query VirusTotal directly (without using the module) to get the **tags**, **reputation**, **last analysis stats** and **alternative names** from certiicates.

In [ ]:
# Code block to query VirusTotal

module_name = "virustotal_public"
attribute_type = "ip-dst"
scan_count = 0

headers = {
    "accept": "application/json",
    "x-apikey": virustotal_apikey
}

print("Start {}.".format(module_name))

for value in query:
    module_comment = "From {} for {}".format(module_name, value)

    if attribute_type in misp_modules[module_name]["input"]:
        data = {
            "attribute": {
                "type": f"{attribute_type}",
                "uuid": str(uuid.uuid4()),
                "value": f"{value}",
            },
            "module": module_name,
            "config": {"apikey": virustotal_apikey}
        }
        print("Query \033[92m{}\033[90m as \033[92m{}\033[90m".format(value, attribute_type))
        result = requests.post("{}/query".format(misp_modules_url), headers=misp_modules_headers, json=data)
        if "results" in result.json() and len(result.json()["results"]) > 0:
            result_json = result.json()["results"]
            for misp_attribute in result_json.get("Attribute", []):
                misp_attribute["comment"] = "{}{}".format(module_comment, misp_attribute.get("comment", ""))
                created_attribute = misp.add_attribute(misp_event.uuid, misp_attribute, pythonify=True)
                if not "errors" in created_attribute:
                    misp.add_object_reference(case_objects[value].add_reference(created_attribute.uuid, "related-to"))
                    print(" Got {} \033[92m{}\033[90m".format(misp_attribute["type"], misp_attribute["value"]))
                    entry = {"source": module_name, "category": misp_attribute["category"], "type": misp_attribute["type"], "enriched": misp_attribute["value"]}
                    playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", misp_attribute["value"])
                else:
                    print(" Unable to add {} \033[92m{}\033[90m to MISP event".format(misp_attribute["type"], misp_attribute["value"]))
            for misp_object in result_json.get("Object", []):
                misp_object["comment"] = "{}{}".format(module_comment, misp_object.get("comment", ""))
                if "Attribute" in misp_object and len(misp_object["Attribute"]) > 0:
                    created_object = misp.add_object(misp_event.uuid, misp_object, pythonify=True)
                    if not "errors" in created_object:
                        for misp_attribute in misp_object.get("Attribute", []):
                            entry = {"source": module_name, "category": misp_attribute["object_relation"], "type": misp_attribute["type"], "enriched": misp_attribute["value"]}
                            playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", misp_attribute["value"])
                        print(" Got \033[92m{}\033[90m ".format(misp_object["name"]))
                        misp.add_object_reference(case_objects[value].add_reference(created_object.uuid, "related-to"))
                    else:
                        print(" Unable to add \033[92m{}\033[90m to MISP event".format(misp_object["name"]))

        #################################################################

        print(" Query VirusTotal directly to get IP details")
        vt_url = "https://www.virustotal.com/api/v3/ip_addresses/{}".format(value)
        result = requests.get(vt_url, headers=headers)
        if "data" in result.json() and len(result.json()["data"]) > 0:
            result_json = result.json()["data"]["attributes"]

            if result_json.get("tags", False):
                print(" Got tags: \033[92m{}\033[90m ".format(result_json.get("tags")))
                entry = {"source": module_name, "category": "tags", "type": "tags", "enriched": result_json.get("tags", [])}
                playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", result_json.get("tags", []))

            if result_json.get("last_analysis_stats", False):
                print(" Got last_analysis_stats: \033[92m{}\033[90m ".format(result_json.get("last_analysis_stats")))
                entry = {"source": module_name, "category": "last_analysis_stats", "type": "last_analysis_stats", "enriched": result_json.get("last_analysis_stats", [])}
                playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", result_json.get("last_analysis_stats", []))

            if result_json.get("reputation", False):
                print(" Got \033[92m{}\033[90m ".format(result_json.get("reputation")))
                entry = {"source": module_name, "category": "reputation", "type": "reputation", "enriched": result_json.get("reputation", 0)}
                playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", result_json.get("reputation", []))

            if result_json.get("last_https_certificate", False):
                subject_alternative_name = result_json.get("last_https_certificate", []).get("extensions", []).get("subject_alternative_name", [])
                for alt_name in subject_alternative_name:
                    print(" Got certificate alt name \033[92m{}\033[90m ".format(alt_name))
                    entry = {"source": module_name, "category": "domain", "type": "domain", "enriched": alt_name}
                    playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", alt_name)
        print(" Finished query VirusTotal directly")

        scan_count += 1
        if scan_count > 5:
            print("Sleeping for {} seconds".format(misp_modules_wait))
            time.sleep(misp_modules_wait)
            scan_count = 0
    else:
        print("Skipping \033[91m{}\033[90m. Not a valid query type ({}).".format(value, misp_modules[module_name]["input"]))

print("Finished {}.".format(module_name))

### VirusTotal enrichment table

The results are now stored in `playbook_results`. Execute the next cell to display them in a table format. The table is also included in the summary for Mattermost and TheHive.

This table returns only those matches corresponding with the source **VirusTotal**.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Category", "Type", "Enriched"]
table.align["Value"] = "l"
table.align["Category"] = "l"
table.align["Type"] = "l"
table.align["Enriched"] = "l"
table._max_width = {"Enriched": 50}
for value in playbook_results:
    for match in playbook_results[value]:
        if match["source"] == "virustotal_public":
            table.add_row([match["source"], value, match["category"], match["type"], match["enriched"]])
print(table.get_string(sortby="Value"))
table_virustotal = table

## ER:4 Enrich with information from Shodan

Use **Shodan** to get more information associated with the IPs defined in `query`, such as **ASN**, **open ports** or discovered **vulnerabilities**.

In [ ]:
# Code block to query Shodan

module_name = "shodan"
attribute_type = "ip-dst"

print("Start {}.".format(module_name))
for value in query:
    module_comment = "From {} for {}".format(module_name, value)

    if attribute_type in misp_modules[module_name]["input"]:
        data = {
            "attribute": {
                "type": f"{attribute_type}",
                "uuid": str(uuid.uuid4()),
                "value": f"{value}",
            },
            "module": module_name,
            "config": {"apikey": shodan_apikey}
        }
        print(" Query \033[92m{}\033[90m as \033[92m{}\033[90m".format(value, attribute_type))
        result = requests.post("{}/query".format(misp_modules_url), headers=misp_modules_headers, json=data)
        if "results" in result.json() and len(result.json()["results"]) > 0:
            result_json = result.json()["results"]
            for misp_object in result_json["Object"]:
                misp_object["comment"] = "{}{}".format(module_comment, misp_object.get("comment", ""))
                if "Attribute" in misp_object and len(misp_object["Attribute"]) > 0:
                    if misp_object["name"] == "ip-api-address":
                        # ip-api-address requires a SRC-IP
                        ip_src_attribute = {
                            "uuid": str(uuid.uuid4()),
                            "object_relation": "ip-src",
                            "value": f"{value}",
                            "type": "ip-src",
                            "disable_correlation": 0,
                            "to_ids": 1,
                            "category": "Network activity"
                        }
                        misp_object["Attribute"].append(ip_src_attribute)
                    created_object = misp.add_object(misp_event.uuid, misp_object, pythonify=True)
                    if not "errors" in created_object:
                        print(" Got \033[92m{}\033[90m ".format(misp_object["name"]))
                        for misp_attribute in misp_object.get("Attribute", []):
                            entry = {"source": module_name, "category": misp_attribute["category"], "type": misp_attribute["type"], "enriched": misp_attribute["value"]}
                            if misp_object["name"] == "ip-api-address":
                                if misp_attribute["object_relation"] == "city":
                                    entry = {"source": module_name, "category": "ip-api-address", "type": "city", "enriched": misp_attribute["value"]}
                                elif misp_attribute["object_relation"] == "ISP":
                                    entry = {"source": module_name, "category": "ip-api-address", "type": "ISP", "enriched": misp_attribute["value"]}
                                elif misp_attribute["object_relation"] == "organization":
                                    entry = {"source": module_name, "category": "ip-api-address", "type": "organization", "enriched": misp_attribute["value"]}
                            playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", misp_attribute["value"])
                            misp.add_object_reference(case_objects[value].add_reference(created_object.uuid, "related-to"))
                    else:
                        print(" Unable to add \033[92m{}\033[90m to MISP event".format(misp_object["name"]))

    else:
        print("Skipping \033[91m{}\033[90m. Not a valid query type ({}).".format(value, misp_modules[module_name]["input"]))

print("Finished {}.".format(module_name))

### Shodan enrichment table

The results are now stored in `playbook_results`. Execute the next cell to display them in a table format. The table is also included in the summary for Mattermost and TheHive.

This table returns only those matches corresponding with the source **Shodan**.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Category", "Type", "Enriched"]
table.align["Value"] = "l"
table.align["Category"] = "l"
table.align["Type"] = "l"
table.align["Enriched"] = "l"
table._max_width = {"Enriched": 50}
for value in playbook_results:
    for match in playbook_results[value]:
        if match["source"] == "shodan":
            table.add_row([match["source"], value, match["category"], match["type"], match["enriched"]])
print(table.get_string(sortby="Value"))
table_shodan = table

## ER:5 Get reputation from Greynoise

Query [Greynoise](https://viz.greynoise.io/) to get the **reputation** of the IPs. The results indicate if the IP is part of [RIOT](https://docs.greynoise.io/docs/riot-data) (*a GreyNoise feature that informs users about IPs used by common business services that are almost certainly not attacking you*) or noise (*when the IP has been observed scanning the internet in the last 90 days*).

In [ ]:
# Code block to query Greynoise

greynoise_url = "https://api.greynoise.io/v3/community/"
headers = {
  'key': greynoise_apikey
}
module_name = "greynoise"
print("Start {}.".format(module_name))
for value in query:
    print("Query \033[92m{}\033[90m".format(value))
    response = requests.request("GET", "{}{}".format(greynoise_url, value), headers=headers)
    if response.status_code == 200:
        result_json = response.json()
        print(" Got \033[92m{}\033[90m ".format(result_json))
        entry = {"source": module_name, "category": "domain", "type": "reputation", "enriched": result_json}
        playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", result_json)

        ip_object = misp.get_object(case_objects[value].uuid, pythonify=True)
        module_comment = "From {} for {}".format(module_name, value)
        ip_object.add_attribute("text", "noise: {}".format(result_json.get("noise")), comment=module_comment, tags=[])
        ip_object.add_attribute("text", "riot: {}".format(result_json.get("riot")), comment=module_comment, tags=[])
        ip_object.add_attribute("text", "classification: {}".format(result_json.get("classification")), comment=module_comment, tags=[])
        ip_object.add_attribute("text", "name: {}".format(result_json.get("name")), comment=module_comment, tags=[])
        ip_object_misp = misp.update_object(ip_object, pythonify=True)
    elif response.status_code == 404:
        print(" Received 404 message: \033[91mIP not found\033[90m.")
    elif response.status_code == 400:
        print(" Received 400 message: \033[91mInvalid IP\033[90m.")
    elif response.status_code == 429:
        print(" Received 429 message: \033[91mRatelimit error\033[90m.")
    else:
        print(" Received \033[91munexpected error\033[90m.")
print("Finished {}.".format(module_name))

## ER:6 Get geo information from MMDB

Add the **geolocation** information from an mmdb server, such as [ip.circl.lu](https://ip.circl.lu/geolookup/).

In [ ]:
# Code block to query MMDB

mmdb_url = "https://ip.circl.lu/geolookup/"
headers = {}
module_name = "mmdb"
print("Start {}.".format(module_name))
for value in query:
    print("Query \033[92m{}\033[90m".format(value))
    response = requests.request("GET", "{}{}".format(mmdb_url, value), headers=headers)
    if response.status_code == 200:
        result_json = response.json()[0]
        iso_code = result_json.get("country", False).get("iso_code", False)
        lat = result_json.get("country_info", False).get("Latitude (average)", False)
        long = result_json.get("country_info", False).get("Longitude (average)", False)

        ip_object = misp.get_object(case_objects[value].uuid, pythonify=True)
        module_comment = "From {} for {}".format(module_name, value)

        if iso_code:
            print(" Got iso_code \033[92m{}\033[90m ".format(iso_code))
            entry = {"source": module_name, "category": "geo", "type": "iso_code", "enriched": iso_code}
            playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", iso_code)
            ip_object.add_attribute("text", "iso_code: {}".format(iso_code), comment=module_comment, tags=[])
        if lat and long:
            print(" Got latitude and longitude \033[92m{}\033[90m \033[92m{}\033[90m".format(lat, long))
            entry = {"source": module_name, "category": "geo", "type": "lat", "enriched": lat}
            playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", lat)
            ip_object.add_attribute("text", "latitude: {}".format(lat), comment=module_comment, tags=[])
            entry = {"source": module_name, "category": "geo", "type": "long", "enriched": long}
            playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", long)
            ip_object.add_attribute("text", "longitude: {}".format(long), comment=module_comment, tags=[])

        ip_object_misp = misp.update_object(ip_object, pythonify=True)
print("Finished {}.".format(module_name))

## ER:7 Enrich with information from AbuseIPDB

Use additional information on the IP reputation from [AbuseIPDB](https://www.abuseipdb.com/).

In [ ]:
# Code block to query AbuseIPDB

abuseipdb_url = "https://api.abuseipdb.com/api/v2/check"
headers = {"Accept": "application/json",
           "Key": abuseipdb_apikey
            }
ipdb_maxage = 90
module_name = "abuseipdb"
print("Start {}.".format(module_name))
for value in query:
    print("Query \033[92m{}\033[90m".format(value))
    querystring = {
        "ipAddress": value,
        "maxAgeInDays": ipdb_maxage
    }
    response = requests.request("GET", url=abuseipdb_url, headers=headers, params=querystring)
    if response.status_code == 200 and "data" in response.json():
        result_json = response.json()["data"]
        abuseConfidenceScore = result_json.get("abuseConfidenceScore")
        print(" Got abuseConfidenceScore \033[92m{}\033[90m".format(abuseConfidenceScore))
        entry = {"source": module_name, "category": "abuseConfidenceScore", "type": "abuseConfidenceScore", "enriched": abuseConfidenceScore}
        playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", abuseConfidenceScore)

        usageType = result_json.get("usageType")
        print(" Got usageType \033[92m{}\033[90m".format(usageType))
        entry = {"source": module_name, "category": "usageType", "type": "usageType", "enriched": usageType}
        playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", usageType)

        totalReports = result_json.get("totalReports")
        print(" Got totalReports \033[92m{}\033[90m".format(totalReports))
        entry = {"source": module_name, "category": "totalReports", "type": "totalReports", "enriched": totalReports}
        playbook_results = pb_add_enrichment(playbook_results, value, entry, "enriched", totalReports)

        ip_object = misp.get_object(case_objects[value].uuid, pythonify=True)
        module_comment = "From {} for {}".format(module_name, value)
        ip_object.add_attribute("text", "abuseConfidenceScore: {}".format(abuseConfidenceScore), comment=module_comment, tags=[])
        ip_object.add_attribute("text", "usageType: {}".format(usageType), comment=module_comment, tags=[])
        ip_object.add_attribute("text", "totalReports: {}".format(totalReports), comment=module_comment, tags=[])
        ip_object_misp = misp.update_object(ip_object, pythonify=True)
print("Finished {}.".format(module_name))

## ER:8 Investigation and enrichment report

The next cell summarises the investigation and enrichment details into a **MISP report**. The report is attached to the MISP event, and also used at the end of the playbook to create the summary.

In [ ]:
summary_iv = "## Investigation and enrichment report\n\n"
current_date = datetime.now()
formatted_date = current_date.strftime("%Y-%m-%d")
virustotal = "virustotal_public"    # put in a variable in case we later want to change hits
for key, sample in playbook_results.items():
    summary_iv += "## Analysis for {}\n".format(key)
    summary_iv += "- Date: **{}**\n".format(formatted_date)
    summary_iv += "- MISP event: **{}** ({})\n".format(misp_event.info, misp_event.id)
    summary_iv += "\n\n"
    summary_iv += "### IP details\n"
    summary_iv += "- ASN: **{}**\n".format(get_playbook_result(key, virustotal, "asn", "AS"))
    summary_iv += "- ASN ownername: **{}**\n".format(get_playbook_result(key, "abuse_finder", "Attribution", "whois-registrant-name"))
    summary_iv += "- Subnet: **{}**\n".format(get_playbook_result(key, virustotal, "subnet-announced", "ip-src"))
    summary_iv += "- ISP: **{}**\n".format(get_playbook_result(key, "shodan", "ip-api-address", "ISP"))
    summary_iv += "- Country: **{}**\n".format(get_playbook_result(key, virustotal, "country", "text"))
    summary_iv += "- City: **{}**\n".format(get_playbook_result(key, "shodan", "ip-api-address", "city"))
    summary_iv += "- Usage: **{}**\n".format(get_playbook_result(key, "abuseipdb", "usageType", "usageType"))
    summary_iv += "- Abuse contact: **{}**\n".format(get_playbook_result(key, "abuse_finder", "Attribution", "whois-registrant-email"))
    summary_iv += "\n\n"
    summary_iv += "### Ratings\n"
    summary_iv += "- VirusTotal detection rate: **{}**\n".format(get_playbook_result(key, virustotal, "detection-ratio", "text"))
    last_analysis_stats = False
    for el in playbook_results[key]:
        if el["source"] == virustotal and el["category"] == "last_analysis_stats" and el["type"] == "last_analysis_stats":
            last_analysis_stats = el["enriched"]
    if last_analysis_stats:
        summary_iv += "     - Harmless: **{}**\n".format(last_analysis_stats["harmless"])
        summary_iv += "     - Malicious: **{}**\n".format(last_analysis_stats["malicious"])
        summary_iv += "     - Suspicious: **{}**\n".format(last_analysis_stats["suspicious"])
        summary_iv += "     - Undetected: **{}**\n".format(last_analysis_stats["undetected"])
        summary_iv += "     - Timeout: **{}**\n".format(last_analysis_stats["timeout"])
    summary_iv += "- AbuseIPDB confidence score: **{}**\n".format(get_playbook_result(key, "abuseipdb", "abuseConfidenceScore", "abuseConfidenceScore"))
    greynoise = False
    for el in playbook_results[key]:
        if el["source"] == "greynoise" and el["category"] == "domain" and el["type"] == "reputation":
            greynoise = el["enriched"]
    if greynoise:
        summary_iv += "- Greynoise:\n"
        summary_iv += "     - Classification: **{}**\n".format(greynoise["classification"])
        summary_iv += "     - In noise?: **{}**\n".format(greynoise["noise"])
        summary_iv += "     - In RIOT?: **{}**\n".format(greynoise["riot"])
    summary_iv += "\n\n"
    summary_iv += "### Associated hostnames\n"
    summary_iv += "- Reverse DNS: **{}**\n".format(get_playbook_result(key, "reversedns", "Network activity", "hostname"))
    summary_iv += "- Hostnames\n"
    for el in get_playbook_result(key, virustotal, "domain", "domain").split(" "):
        summary_iv += "     - {}\n".format(el)
    for el in get_playbook_result(key, "shodan", "Network activity", "domain").split(" "):
        summary_iv += "     - {}\n".format(el)
    summary_iv += "\n\n"
    vulnerability = get_playbook_result(key, "shodan", "External analysis", "vulnerability")
    if vulnerability and len(vulnerability) > 1:
        summary_iv += "### Associated vulnerabilities\n"
        summary_iv += "- **{}**\n".format(vulnerability)
        summary_iv += "\n\n"

    ports = get_playbook_result(key, "shodan", "Network activity", "port")
    if ports and len(ports) > 1:
        summary_iv += "### Associated open ports\n"
        for el in ports.split(" "):
            summary_iv += "- **{}**\n".format(el)
        summary_iv += "\n\n"
 
event_title = "IP reputation"
print("Creating MISP report \033[92m{}\033[90m".format(event_title))
chunk_size = 61500
for i in range(0, len(summary_iv), chunk_size):
    chunk = summary_iv[i:i + chunk_size]
    event_report = MISPEventReport()
    event_title_edit = event_title
    if i > 0:
        event_title_edit = "{} ({} > {})".format(event_title, i, i + chunk_size)
    event_report.name = event_title_edit
    event_report.content = chunk
    result = misp.add_event_report(misp_event.id, event_report)
    if "EventReport" in result:
        print(" Report ID: \033[92m{}\033[90m".format(result.get("EventReport", []).get("id", 0)))
    else:
        print("Failed to create report for \033[91m{}\033[90m.".format(event_title))

# Closure

In this **closure** or end step we create a **summary** of the actions that were performed by the playbook. The summary is printed in the playbook and can also be send to a chat channel. 

## EN:1 MISP indicators

The next section first **queries MISP for the indicators added to the MISP event** that is linked to the execution of this playbook.

The indicators are stored in the variable `indicator_table` (table format) and `indicator_raw_list` (in raw format) which is used in a later section to create the playbook summary.

In [ ]:
# Get all the indicators for our event and store this is in a table. We can also use this for the summary.
indicator_search = misp.search("attributes", uuid=misp_event.uuid, to_ids=True, pythonify=True)
indicator_raw_list = []
indicator_table = PrettyTable()
if len(indicator_search) > 0:
    indicator_table.field_names = ["Type", "Category", "Indicator", "Comment"]
    indicator_table.align["Type"] = "l"
    indicator_table.align["Category"] = "l"
    indicator_table.align["Indicator"] = "l"
    indicator_table.align["Comment"] = "l"
    indicator_table.border = True
    for indicator in indicator_search:
        if indicator.value not in indicator_raw_list:
            indicator_table.add_row([indicator.type, indicator.category, indicator.value, indicator.comment])
            indicator_raw_list.append(indicator.value)
    print("Got \033[92m{}\033[90m indicator(s) from the event \033[92m{}\033[90m ({}).\n\n".format(len(indicator_raw_list), misp_event.info, misp_event.id))
else:
    print("\033[93mNo indicators found in the event \033[92m{}\033[90m ({})".format(misp_event.info, misp_event.id))

### Raw list of MISP indicators

The indicators are now stored in `indicator_search` (as Python objects) and `indicator_raw_list` (in raw format, only the indicators). Execute the next cell to display them in a table format. The table is also included in the summary sent to Mattermost and TheHive.

In [ ]:
if len(indicator_raw_list) > 0:
    print(indicator_table.get_string(sortby="Type"))
    print("\n\nIndicator list in raw format:")
    print("---------------------------------------------------")
    for el in indicator_raw_list:
        print("{}".format(el))
    print("---------------------------------------------------")

## EN:2 Create the summary of the playbook

The next section creates a summary and stores the output in the variable `summary` in Markdown format. It also stores an intro text in the variable `intro`. These variables are later used when sending information to Mattermost or TheHive.

In [ ]:
summary = "# MISP Playbook summary\nQuery IP reputation with MISP event: **{}** ({}/events/view/{}). ".format(misp_event.info, misp_url, misp_event.id)

summary += "\n"
summary += summary_iv
summary += "\n"
intro = summary

summary += "## Indicators\n\n"
summary += "### Indicators table\n\n"
if len(indicator_raw_list) > 0:
    indicator_table.set_style(MARKDOWN)
    summary += indicator_table.get_string(sortby="Type")
    summary += "\n\n\n"
    summary += "### Indicators in **raw format**\n\n"
    for indicator in indicator_raw_list:
        summary += "{}\n\n".format(indicator)
    summary += "\n"
else:
    summary += "There are no indicators"
summary += "\n\n"

summary += "## Correlations\n\n"
summary += "### MISP event matches\n\n"
table_mispevents.set_style(MARKDOWN)
summary += table_mispevents.get_string()
summary += "\n\n"

summary += "### MISP feed matches\n\n"
table_mispfeeds.set_style(MARKDOWN)
summary += table_mispfeeds.get_string()
summary += "\n\n"

summary += "\n\n"

print("The \033[92msummary\033[90m of the playbook is available.\n")

## EN:3 Print the summary

Apart from the full summary of the investigation and enrichment, the IP triage summary provides the necessary input for the analyst. This summary was previously also added as a MISP report.

In [ ]:
# Comment the printing of the summary to avoid cluttering the playbook by accident
#display_markdown(summary, raw=True)

## EN:4 Display a map of the IPs

Now use folium to render the IPs on a world map. The geographical locations are obtained from the MMDB server.

In [ ]:
world_map = folium.Map(location=[0, 0], zoom_start=2)

for value in query:
    lat = get_playbook_result(value, "mmdb", "geo", "lat")
    long = get_playbook_result(value, "mmdb", "geo", "long")
    
    marker = folium.Marker(location=[lat, long], popup=value)
    marker.add_to(world_map)
    
world_map

## EN:5 Send a summary to Mattermost

Now you can send the summary to Mattermost. You can send the summary in two ways by selecting one of the options for the variable `send_to_mattermost_option` in the next cell.

- The default option where the entire summary is in the **chat**, or
- a short intro and the summary in a **card**

For this playbook we rely on a webhook in Mattermost. You can add a webhook by choosing the gear icon in Mattermost, then choose Integrations and then **Incoming Webhooks**. Set a channel for the webhook and lock the webhook to this channel with *"Lock to this channel"*.

In [ ]:
send_to_mattermost_option = "via a chat message"
#send_to_mattermost_option = "via a chat message with card"

In [ ]:
message = False
if send_to_mattermost_option == "via a chat message":
    message = {"username": mattermost_playbook_user, "text": summary}
elif send_to_mattermost_option == "via a chat message with card":
    message = {"username": mattermost_playbook_user, "text": intro, "props": {"card": summary}}

if message:
    r = requests.post(mattermost_hook, data=json.dumps(message))
    r.raise_for_status()
if message and r.status_code == 200:
    print("Summary is \033[92msent to Mattermost.\n")
else:
    print("\033[91mFailed to sent summary\033[90m to Mattermost.\n")

## EN:6 Send an alert to TheHive

Next to informing your colleagues via Mattermost you can also send an **alert** to TheHive. The alert contains the summary, and a list of indicators as 'observables'.

You can change the alert title with `thehive_alert_title` and provide a reference type with `thehive_alert_reference`. Note that this reference needs to be **unique** in TheHive. If you want to create multiple alerts for the same MISP event then add some random value at the end.

In [ ]:
# The title of the TheHive alert
thehive_alert_title = "MISP Playbook Summary - Query IP reputation"

# A unique reference for the TheHive (we include the MISP event UUID)
thehive_alert_reference = "MISP event - {} - {}".format(misp_event.info, misp_event.uuid)

# Alert type in TheHive
thehive_alert_type = "MISP Playbook alert"

# TLP:Amber for TheHive
thehive_tlp = 2

# PAP:GREEN for TheHive
thehive_pap = 1

In [ ]:
# Code block to send an alert to TheHive
# We use the Python requests library
thehive_headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {thehive_key}'}
thehive_url_create = "{}/api/v1/alert".format(thehive_url)

thehive_observables = []

for value in indicator_raw_list:
    attribute_type = False
    for expr in regular_expressions:
        if re.match(r"{}".format(regular_expressions[expr]), value):
            attribute_type = expr
            break
    if attribute_type:
        if attribute_type == "ip-src" or attribute_type == "ip-dst":
            dataType = "ip"
        elif attribute_type == "md5" or attribute_type == "sha256" or attribute_type == "sha1":
            dataType = "hash"
        else:
            dataType = attribute_type

        if dataType:
            thehive_observables.append({"dataType": dataType, "data": value, "pap": thehive_pap, "tlp": thehive_tlp})

thehive_alert = {"title": thehive_alert_title,
                 "description": intro,
                 "summary": summary[0:1048576],
                 "type": thehive_alert_type,
                 "source": "playbook",
                 "sourceRef": thehive_alert_reference,
                 "tlp": thehive_tlp, "pap": thehive_pap,
                 "observables": thehive_observables}

result = requests.post(thehive_url_create, headers=thehive_headers, data=json.dumps(thehive_alert))
if result.status_code == 201 and result.json()['status'] == 'New':
    thehive_alert_id = result.json()['_id']
    print('The TheHive \033[92malert {} is added'.format(thehive_alert_id))
else:
    print('\033[91mFailed\033[90m to add TheHive alert')
    print(result.text)

## EN:7 End of the playbook 

In [ ]:
print("\033[92m End of the playbook")


## External references <a name="extreferences"></a>

- [The MISP Project](https://www.misp-project.org/)
- [Mattermost](https://mattermost.com/)
- [TheHive](https://thehive-project.org/)
- [ip.circl.lu](https://ip.circl.lu/geolookup/)
- [abuse_finder](https://github.com/certsocietegenerale/abuse_finder)
- [AbuseIPDB](https://www.abuseipdb.com/)

## Technical details 

### Documentation

This playbook requires these Python **libraries** to exist in the environment where the playbook is executed. You can install them with `pip install <library>`.

```
pyfaup
chardet
PrettyTable
ipywidgets
mattermostdriver
folium

```

### Colour codes

The output from Python displays some text in different colours. These are the colour codes

```
Red = '\033[91m'
Green = '\033[92m'
Blue = '\033[94m'
Cyan = '\033[96m'
White = '\033[97m'
Yellow = '\033[93m'
Magenta = '\033[95m'
Grey = '\033[90m'
Black = '\033[90m'
Default = '\033[99m'
```